The Stanford Open Policing Datasets provide information on traffic stops in different states. Please download the csv files for Montana and Vermont. You may also find the README file helpful as you work through the challenge questions. It contains useful information about the dataset, including column descriptions.

In [102]:
#import dependencies
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

from math import pi, cos
from sklearn import linear_model

In [12]:
#read data
MT = pd.read_csv('MT_cleaned.csv', low_memory=False) # montana
VT = pd.read_csv('VT_cleaned.csv',  low_memory=False) # vermont

**What proportion of traffic stops in Montana involved male drivers? In other words, divide the number of traffic stops involving male drivers by the total number of stops.**

In [13]:
#Explore Gender Data
byGender = MT.groupby('driver_gender')
byGenderKeys = byGender.groups.keys()
print(byGenderKeys)
genderCount = byGender.apply(len)
print(genderCount)

dict_keys(['F', 'M'])
driver_gender
F    268065
M    556934
dtype: int64


In [14]:
#problem 1
gCount = genderCount.values
maleProportion = gCount[1]/(gCount[0]+gCount[1])
print("Proportion of traffic stops in Montana involving Male Drivers: ")
maleProportion

Proportion of traffic stops in Montana involving Male Drivers: 


0.6750723334210103

**How many more times likely are you to be arrested in Montana during a traffic stop if you have out of state plates?**

In [18]:
#problem 2
#Explore Out of State Data
byOutOfState = MT.groupby(['is_arrested', 'out_of_state'])
byOutOfStateKeys = byOutOfState.groups.keys()
print(byOutOfStateKeys)
#There are some 'nan' values that we must ignore for out of state
OutOfStateCount = byOutOfState.apply(len)
OutOfStateCount

dict_keys([(False, True), (False, False), (True, nan), (True, False), (False, nan), (True, True)])


is_arrested  out_of_state
False        False           604588
             True            198773
True         False            12190
             True              4868
dtype: int64

In [27]:
#We need to find probability of being arrested, given out of state plates: 
#And probability of being arrested, given in-state plates:
#i.e. P(Arrested | Out of State) = P(Arrested and Out of State)/P(Out of State)
#and P(Arrested | In-State) = P(Arrested and In-State)/P(In-State)
OoSCount = OutOfStateCount.values
#print(OoSCount)
total = OutOfStateCount.values.sum()
#print(total)
P_A_given_OoS = OoSCount[3]/(OoSCount[1] + OoSCount[3])
P_A_given_IS  = OoSCount[2]/(OoSCount[0] + OoSCount[2])
print("Ratio of being arrested w/ Out of State plates vs in-state plates: ")
prob = P_A_given_OoS/P_A_given_IS
prob

Ratio of being arrested w/ Out of State plates vs in-state plates: 


1.2095129351452942

**Perform a (χ2) test to determine whether the proportions of arrests in these two populations are equal. What is the value of the test statistic?**

In [34]:
#problem 3
#Explore Arrest Data for both States
mtArrest = MT.is_arrested.value_counts()
print(mtArrest)

print("-"*70)

vtArrest = VT.is_arrested.value_counts()
print(vtArrest)

False    807923
True      17195
Name: is_arrested, dtype: int64
----------------------------------------------------------------------
False    279954
True       3331
Name: is_arrested, dtype: int64


**Create Cross Table of Observed Frequencies:**

|  || Montana | Vermont |
| --- | --- | --- |
| Arrested || 17195 | 3331 |
| Not Arrested || 807923 | 279954 |

Now, follow the method outlined in http://www.stat.wmich.edu/s216/book/node115.html for chi-squared statistic. <br> <br>
We are nterested in comparing arrest rate in Montana vs arrest rate in Vermont.The null hypothesis can be written as: <br> <br> 
<center>$H_0: p_1  = p_2$ <br>
vs <br>
$H_1: p_1 \neq p_2$ <br> <br></center>
Find the Expected values and then use them with observed in following formula: <br> <br>
<center>$\chi^2=\sum_{k=1}^{n} \frac{(O_k - E_k)^2}{E_k}$</center>

In [50]:
A_m, nA_m = MT.is_arrested.value_counts()
A_v, nA_v = VT.is_arrested.value_counts()
N = MT.is_arrested.count() + VT.is_arrested.count()
#create contingency table
conTbl = np.array([[A_m, A_v], [nA_m, nA_v]])
chi2, p, dof, ex = stats.chi2_contingency(conTbl)
print("Chi-squared statistic value is: ")
chi2

Chi-squared statistic value is: 


956.2921877903868

**What proportion of traffic stops in Montana resulted in speeding violations? In other words, find the number of violations that include "Speeding" in the violation description and divide that number by the total number of stops (or rows in the Montana dataset).**

In [69]:
#problem 04
byViolation = MT.groupby('violation')
vTotal_mt = MT.violation.count()
vCount = MT.violation.str.contains("Speeding|speeding").sum()
speedProportion = vCount/(vTotal_mt)
print("Proportion of stops that resulted in speeding violations: ")
speedProportion

Proportion of stops that resulted in speeding violations: 


0.6581580398644923

**How much more likely does a traffic stop in Montana result in a DUI than a traffic stop in Vermont? To compute the proportion of traffic stops that result in a DUI, divide the number of stops with "DUI" in the violation description by the total number of stops.**

In [75]:
#problem 05
duiCount_mt = MT.violation.str.contains("DUI").sum()
duiProportion_mt = duiCount_mt/(vTotal_mt)

byViolation = VT.groupby('violation')
vTotal_vt = VT.violation.count()
duiCount_vt = VT.violation.str.contains("DUI").sum()
duiProportion_vt = duiCount_vt/(vTotal_vt)

duiMoreLikely = duiProportion_mt/duiProportion_vt
print("Likelihood of proportion of stops with DUI in MT vs VT: ")
duiMoreLikely

Likelihood of proportion of stops with DUI in MT vs VT: 


4.054943765214862

**What is the extrapolated, average manufacture year of vehicles involved in traffic stops in Montana in 2020? To answer this question, calculate the average vehicle manufacture year for each year's traffic stops. Extrapolate using a linear regression.**

In [76]:
#problem 6
df = MT[['stop_date','vehicle_year']]
df["stop_date"] = pd.to_datetime(df["stop_date"]).dt.year

/home/usama/.virtualenvs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [85]:
#Explore Data for Vehicle Year
byVehicleDate = df.groupby('vehicle_year')
#print(byVehicleDate.count())
byVehicleDateKeys = byVehicleDate.groups.keys()
#print(byVehicleDateKeys)

In [86]:
#remove NON and UNK and get X and Y data for regression
df = df[~df['vehicle_year'].isin(['NON-', 'UNK'])]
df['vehicle_year'] = df['vehicle_year'].astype('float64') 
byVehicleDate = df.groupby('vehicle_year')
#print(byVehicleDate.count())
byStopDate = df.groupby('stop_date', as_index=False).agg({"vehicle_year": "mean"})
print(byStopDate.head())
#print(type(byStopDate))
X = byStopDate['stop_date'].values
X = X[:,None]
#print(X)
Y = byStopDate['vehicle_year'].values

#print(Y)

   stop_date  vehicle_year
0     2009.0   2000.980215
1     2010.0   2001.521377
2     2011.0   2002.280938
3     2012.0   2003.362207
4     2013.0   2003.905175


In [89]:
#regression
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
#predictions = model.predict(X)
predictions = lm.predict(2020)
print(model.intercept_, model.coef_)
print("-"*70)
print("Prediction for 2020: ")
predictions

559.661028049426 [0.7174169]
----------------------------------------------------------------------
Prediction for 2020: 


array([2008.84316596])

**What is the p-value of this linear regression?**

In [92]:
#problem 7
X2 = sm.add_constant(X)
model = sm.OLS(Y, X2).fit()
#print(model.summary())
#print("-"*70)
print("p-value: ")
model.pvalues[0]

p-value: 


1.5226470572045793e-05

**Combining both the Vermont and Montana datasets, find the hours when the most and least number of traffic stops occurred. What is the difference in the total number of stops that occurred in these two hours? Hours range from 00 to 23. Round stop times down to compute this difference.**

In [94]:
#problem 8
MT['hour'] = pd.to_datetime(MT['stop_time'], format='%H:%M').dt.hour
VT['hour'] = pd.to_datetime(VT['stop_time'], format='%H:%M').dt.hour
byHourMT = MT[['hour','stop_time']]
byHourVT = VT[['hour','stop_time']]

byHour = pd.concat([byHourMT, byHourVT])
#byHour =byHour.groupby('hour')
#print(byHour.count())
byHour = byHour.groupby('hour', as_index=False).agg({"stop_time": "count"})
data = byHour.values
hours = data[:,0]
counts = data[:,1]
maxValue = np.max(counts)
maxIdx = np.argmax(counts)
#print("Max Value: ", maxValue)
#print("Max Hour: ", hours[maxIdx])
minValue = np.min(counts)
minIdx = np.argmin(counts)
#print("Min Value: ", minValue)
#print("Min Hour: ", hours[minIdx])
#print("-"*70)
print("total num of stops: ")
maxValue-minValue

total num of stops: 


95344.0

**We can use the traffic stop locations to estimate the areas of the counties in Montana. Represent each county as an ellipse with semi-axes given by a single standard deviation of the longitude and latitude of stops within that county. What is the area, in square kilometers, of the largest county measured in this manner? Please ignore unrealistic latitude and longitude coordinates.**

In [107]:
forEllipse = MT[['county_fips', 'county_name', 'lon', 'lat']]
#print(forEllipse.head())

In [108]:
#print(forEllipse.county_name.unique())

In [109]:
#print(forEllipse.groupby('county_name').count())

Montana, USA Lat Long Coordinates Info:<br> The latitude of Montana, USA is 46.965260, and the longitude is -109.533691.

In [110]:
forEllipse = forEllipse[np.isfinite(forEllipse['lon'])]
forEllipse = forEllipse[np.isfinite(forEllipse['lat'])]
#print(forEllipse.count())

In [124]:
byCounty = forEllipse.groupby('county_fips', as_index=False).agg({"county_name": 'max', "lon": "std", "lat": "std"})
print(byCounty.head())

   county_fips        county_name       lon       lat
0      30001.0  Beaverhead County  0.185625  0.268133
1      30003.0    Big Horn County  1.937072  0.112396
2      30005.0      Blaine County  0.258872  0.042086
3      30007.0  Broadwater County  7.825885  3.248041
4      30009.0      Carbon County  0.164160  0.150638


Each degree of latitude is approximately 111.321 kilometers apart. <br>
1 degree of longitude = cosine(latitude in degrees) x length of degree at equator(~111.321km)
<br>
**Source:** <br>
https://www.colorado.edu/geography/gcraft/warmup/aquifer/html/distance.html <br>


Area of an Ellipse = $\pi*a*b$ <br>
where, a and b are the lengths of the semi-minor and semi-major axes

In [129]:
byCounty["lat_km"] = byCounty["lat"]*(pi/180)*111.321 
Lati = 46.96
cosLati = cos(Lati*(pi/180))
byCounty["lon_km"] = byCounty["lon"]*(pi/180)*111.321*cosLati
byCounty["area"] = byCounty["lat_km"]*byCounty["lon_km"]*pi
print()
max_area = byCounty['area'].max()
max_loc = byCounty['area'].idxmax()
county_max = byCounty.county_name[max_loc]
print("County with Max area from number of traffic stops: ",'\n')
print(county_max)
max_area


County with Max area from number of traffic stops:  

Broadwater County


205.74155905010772